In [2]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
import toolz

import os

def custom(data):
    return toolz.curried.pipe(data, alt.to_json(filename='altdata/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('custom', custom)
alt.data_transformers.enable('custom')

pass

In [14]:
df = pd.read_csv("dpt2020.csv", sep=";")
df.drop(df[df.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df.drop(df[df.dpt == 'XX'].index, inplace=True)
df['annais'] = df['annais'].astype(int)
df['nombre'] = df['nombre'].astype(int)
df['sexe'] = df['sexe'].replace({1: 'M', 2: 'F'})
df = df.rename(columns={"annais": "annee"})

In [4]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

# Aggréger les données par prénom et sexe
df_aggregated = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nlargest(10, 'nombre')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nlargest(10, 'nombre')

top_10_boys = top_10_boys.sort_values(by='nombre', ascending=False)
top_10_girls = top_10_girls.sort_values(by='nombre', ascending=False)

# Créer une liste vide pour stocker les données alternées
alternated_list = []

# Utiliser une boucle pour ajouter alternativement les prénoms des garçons et des filles
for i in range(max(len(top_10_boys), len(top_10_girls))):
    alternated_list.append(top_10_boys.iloc[i])
    alternated_list.append(top_10_girls.iloc[i])

# Convertir la liste alternée en DataFrame
df_alternated = pd.DataFrame(alternated_list)

# Création de l'histogramme avec Altair
chart = alt.Chart(df_alternated).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom', sort=None),
    y=alt.Y('nombre:Q', title='Nombre d\'enfants'),
    color=alt.Color('sexe:N', scale=alt.Scale(domain=['M', 'F'], range=['blue', 'pink']), title='Sexe'),
    xOffset=alt.XOffset("sexe:N")
).properties(
    width=800  # pour ajuster la largeur des barres
)

chart

alt.Chart(...)

In [5]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

# Aggréger les données par prénom et sexe
df_aggregated = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nlargest(10, 'nombre')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nlargest(10, 'nombre')

top_10_boys = top_10_boys.sort_values(by='nombre', ascending=False)
top_10_girls = top_10_girls.sort_values(by='nombre', ascending=False)

top_10_boys['nombre'] = top_10_boys['nombre'] / top_10_boys['nombre'].iloc[0]
top_10_girls['nombre'] = top_10_girls['nombre'] / top_10_girls['nombre'].iloc[0]

# Créer une liste vide pour stocker les données alternées
alternated_list = []

# Utiliser une boucle pour ajouter alternativement les prénoms des garçons et des filles
for i in range(max(len(top_10_boys), len(top_10_girls))):
    alternated_list.append(top_10_boys.iloc[i])
    alternated_list.append(top_10_girls.iloc[i])

# Convertir la liste alternée en DataFrame
df_alternated = pd.DataFrame(alternated_list)

# Création de l'histogramme avec Altair
chart = alt.Chart(df_alternated).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom', sort=None),
    y=alt.Y('nombre:Q', title='Nombre d\'enfants'),
    color=alt.Color('sexe:N', scale=alt.Scale(domain=['M', 'F'], range=['blue', 'pink']), title='Sexe'),
    xOffset=alt.XOffset("sexe:N")
).properties(
    width=800  # pour ajuster la largeur des barres
)

chart

alt.Chart(...)

In [24]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

number_of_names = 1000

# Aggréger les données par prénom et sexe
df_aggregated = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nlargest(number_of_names, 'nombre')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nlargest(number_of_names, 'nombre')

top_10_boys = top_10_boys.sort_values(by='nombre', ascending=False)
top_10_girls = top_10_girls.sort_values(by='nombre', ascending=False)

top_10_boys['nombre'] = top_10_boys['nombre'] / top_10_boys['nombre'].iloc[0]
top_10_girls['nombre'] = top_10_girls['nombre'] / top_10_girls['nombre'].iloc[0]

# creating data 
df_overlap = pd.DataFrame({'Boys': top_10_boys['nombre'].tolist(), 
                   'Girls': top_10_girls['nombre'].tolist(),
                   'x': list(range(1, number_of_names + 1))})

# Overlapping Histograms 
alt.Chart(df_overlap).transform_fold( 
    ['Boys', 'Girls'], 
    as_=['Columns', 'Values'] 
).mark_area( 
    opacity=0.5, 
    interpolate='step'
).encode( 
    alt.X('x:Q', title='Rank'), 
    alt.Y('Values:Q', stack=None), 
    alt.Color('Columns:N') 
).properties(
    width=800  # pour ajuster la largeur des barres
)



alt.Chart(...)

In [25]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

number_of_names = 1000

# Aggréger les données par prénom et sexe
df_aggregated = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nlargest(number_of_names, 'nombre')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nlargest(number_of_names, 'nombre')

top_10_boys = top_10_boys.sort_values(by='nombre', ascending=False)
top_10_girls = top_10_girls.sort_values(by='nombre', ascending=False)

top_10_boys['nombre'] = top_10_boys['nombre'] / top_10_boys['nombre'].iloc[0]
top_10_girls['nombre'] = top_10_girls['nombre'] / top_10_girls['nombre'].iloc[0]

# creating data 
df_overlap = pd.DataFrame({'Boys': top_10_boys['nombre'].tolist(), 
                   'Girls': top_10_girls['nombre'].tolist(),
                   'x': list(range(1, number_of_names + 1))})

# Overlapping Histograms 
alt.Chart(df_overlap).transform_fold( 
    ['Boys', 'Girls'], 
    as_=['Columns', 'Values'] 
).mark_area( 
    opacity=0.5, 
    interpolate='step'
).encode( 
    alt.X('x:Q', title='Rank'), 
    alt.Y('Values:Q', stack=None, scale=alt.Scale(type='log')), 
    alt.Color('Columns:N') 
).properties(
    width=800  # pour ajuster la largeur des barres
)

alt.Chart(...)